In [1]:
"""
Name: Jiajun Bao
CSE 163 Final Project
This program analyzes the relationship between features of light-duty vehicles
such as engine size, vehicle classification, number of cylinders, etc., and
their carbon dioxide emissions using Pandas and Altair. Then the program
utilizes machine learning Sklearn library to predict carbon dioxide emissions
for new light-duty vehicles and analyze and interpret which features are the
most informative for how a decision is made in the machine learning model.

Steps to run the code:
(1) install any of the following libraries if haven't so by typing the
    following command: pip install (library name). There should be no error
    messages if all the required libraries have been installed.
(2) The data set 2021_Fuel_Consumption.csv should have been downloaded and
    avalibale under the same folder of this notebook.If not, the dataset can
    be accessed at: https://www.nrcan.gc.ca/sites/nrcan/files/oee/files/csv/     
    MY2021%20Fuel%20Consumption%20Ratings.csv and the data file needs to be 
    saved in the same fodler as the Jupyter Notebook.
(3) Then run the Python code to in the order of the current Jupyter Notebook.
    Wrong order may lead to unexpected errors. Please contact me at 
    Jiajunb@uw.edu if you have any further questions.
""" 
import pandas as pd
import altair as alt
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [2]:
# Load in the data file
file_path = '2021_Fuel_Consumption.csv'
df_raw = pd.read_csv(file_path, low_memory=False)
print('-----Raw Data-----')
print(df_raw.columns)
df_raw

-----Raw Data-----
Index(['Model', 'Make', 'Model.1', 'Vehicle Class', 'Engine Size', 'Cylinders',
       'Transmission', 'Fuel', 'Fuel Consumption', 'Unnamed: 9',
       ...
       'Unnamed: 211', 'Unnamed: 212', 'Unnamed: 213', 'Unnamed: 214',
       'Unnamed: 215', 'Unnamed: 216', 'Unnamed: 217', 'Unnamed: 218',
       'Unnamed: 219', 'Unnamed: 220'],
      dtype='object', length=221)


,Model,Make,Model.1,Vehicle Class,Engine Size,Cylinders,Transmission,Fuel,Fuel Consumption,Unnamed: 9,...,Unnamed: 211,Unnamed: 212,Unnamed: 213,Unnamed: 214,Unnamed: 215,Unnamed: 216,Unnamed: 217,Unnamed: 218,Unnamed: 219,Unnamed: 220
0,Year,NaN,NaN,NaN,(L),NaN,NaN,Type,City (L/100 km),Hwy (L/100 km),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,Acura,ILX,Compact,2.4,4.0,AM8,Z,9.9,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,Acura,NSX,Two-seater,3.5,6.0,AM9,Z,11.1,10.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,Acura,RDX SH-AWD,SUV: Small,2.0,4.0,AS10,Z,11.0,8.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,Acura,RDX SH-AWD A-SPEC,SUV: Small,2.0,4.0,AS10,Z,11.3,9.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Preparing the Data for Later Analysis. The cleaning process isput in a
# function so testing purpose
				

def clean_data(df):
    """
    This function takes a Pandas DataFrame that represent orginal dataset
    of the 2021 Fuel Consumption Ratings and returned cleaned data for
    later analysis.
    """
    # rename some column names and combine the first two rows of the original
    # data as both contain column names
    df.rename(columns={'Model': 'Year'}, inplace=True)
    df.rename(columns={'Make': 'Brand'}, inplace=True)
    df.rename(columns={'Model.1': 'Model'}, inplace=True)
    df.rename(columns={'Fuel': 'Fuel Type'}, inplace=True)
    df.rename(columns={'Engine Size': 'Engine Size(L)'}, inplace=True)
    df.rename(columns={'Fuel Consumption': 'Fuel Consumption City(L/100 km)'},
              inplace=True)
    df.rename(columns={'Unnamed: 9': 'Fuel Consumption Hwy(L/100 km)'},
              inplace=True)
    df.rename(columns={'Unnamed: 10': 'Fuel Consumption Comb(L/100 km)'},
              inplace=True)
    df.rename(columns={'CO2 Emissions': 'CO2 Emissions(g/km)'}, inplace=True)
    df.rename(columns={'CO2': 'CO2 Rating'}, inplace=True)
    df.rename(columns={'Smog': 'Smog Rating'}, inplace=True)

    # drop 2nd row after combining all info with first two rows
    df = df.drop(df.index[0])

    # filter columns to relevant ones and get rid of empty columns/rows
    df = df[['Year', 'Brand', 'Model', 'Vehicle Class', 'Engine Size(L)',
            'Cylinders', 'Transmission', 'Fuel Type', 'CO2 Rating',
            'Smog Rating', 'Fuel Consumption City(L/100 km)',
            'Fuel Consumption Hwy(L/100 km)',
            'Fuel Consumption Comb(L/100 km)', 'CO2 Emissions(g/km)']]

    df = df.dropna()

    # create a new variable contain both brand and model for later plotting
    df['Brand & Model'] = df['Brand'] + '-' + df['Model']

    # reset the index starting from 0
    df = df.reset_index(drop=True)

    # change all data entries for CO2 Emissions and Fuel Consumption to
    # numeric since some entries are in string type
    df['CO2 Emissions(g/km)'] = pd.to_numeric(df['CO2 Emissions(g/km)'])
    df['Fuel Consumption City(L/100 km)'] = pd.to_numeric(
        df['Fuel Consumption City(L/100 km)'])
    df['Fuel Consumption Hwy(L/100 km)'] = pd.to_numeric(
        df['Fuel Consumption Hwy(L/100 km)'])
    df['Fuel Consumption Comb(L/100 km)'] = pd.to_numeric(
        df['Fuel Consumption Comb(L/100 km)'])
    return df


df = clean_data(df_raw)
print()
print('-----After Cleaning-----')
print(df.columns)
df


-----After Cleaning-----
Index(['Year', 'Brand', 'Model', 'Vehicle Class', 'Engine Size(L)',
       'Cylinders', 'Transmission', 'Fuel Type', 'CO2 Rating', 'Smog Rating',
       'Fuel Consumption City(L/100 km)', 'Fuel Consumption Hwy(L/100 km)',
       'Fuel Consumption Comb(L/100 km)', 'CO2 Emissions(g/km)',
       'Brand & Model'],
      dtype='object')


,Year,Brand,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,CO2 Rating,Smog Rating,Fuel Consumption City(L/100 km),Fuel Consumption Hwy(L/100 km),Fuel Consumption Comb(L/100 km),CO2 Emissions(g/km),Brand & Model
0,2021,Acura,ILX,Compact,2.4,4.0,AM8,Z,6,3,9.9,7.0,8.6,199,Acura-ILX
1,2021,Acura,NSX,Two-seater,3.5,6.0,AM9,Z,4,3,11.1,10.8,11.0,256,Acura-NSX
2,2021,Acura,RDX SH-AWD,SUV: Small,2.0,4.0,AS10,Z,5,6,11.0,8.6,9.9,232,Acura-RDX SH-AWD
3,2021,Acura,RDX SH-AWD A-SPEC,SUV: Small,2.0,4.0,AS10,Z,5,6,11.3,9.1,10.3,242,Acura-RDX SH-AWD A-SPEC
4,2021,Acura,TLX SH-AWD,Compact,2.0,4.0,AS10,Z,5,7,11.2,8.0,9.8,230,Acura-TLX SH-AWD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,2021,Volvo,XC40 T5 AWD,SUV: Small,2.0,4.0,AS8,Z,5,5,10.7,7.7,9.4,219,Volvo-XC40 T5 AWD
879,2021,Volvo,XC60 T5 AWD,SUV: Small,2.0,4.0,AS8,Z,5,5,11.1,8.3,9.9,230,Volvo-XC60 T5 AWD
880,2021,Volvo,XC60 T6 AWD,SUV: Small,2.0,4.0,AS8,Z,5,7,11.7,8.6,10.3,240,Volvo-XC60 T6 AWD
881,2021,Volvo,XC90 T5 AWD,SUV: Standard,2.0,4.0,AS8,Z,5,5,11.5,8.4,10.1,236,Volvo-XC90 T5 AWD


In [4]:
def scatter_plot(independent_var, dependent_var):
    """
    The function takes two Pandas Series independent_var and dependent_var
    and plots(returns) a scatter plot of the relationship between the
    independent variable and dependent variable.
    """
    # Brush for selection
    brush = alt.selection(type='interval')

    # Scatter Plot
    points = alt.Chart(df).mark_point().encode(
        x=alt.X(independent_var),
        y=alt.Y(dependent_var, sort='ascending', axis=alt.Axis(tickCount=10)),
        color=alt.condition(brush, independent_var, alt.value('lightgray'),
                            legend=None),
        tooltip='Brand & Model'
    ).add_selection(brush).properties(
        title='Relationship between ' + independent_var + ' and '
              + dependent_var,
        width=500,
        height=300
    )

    # Base chart for data tables
    ranked_text = alt.Chart(df).mark_text().encode(
        y=alt.Y('row_number:O', axis=None)
    ).transform_window(
        row_number='row_number()'
    ).transform_filter(
        brush
    ).transform_window(
        rank='rank(row_number)'
    ).transform_filter(
        alt.datum.rank < 20
    )

    # Data Tables
    x_var = ranked_text.encode(text=independent_var).properties(
        title=independent_var)
    y_var = ranked_text.encode(text=dependent_var).properties(
        title=dependent_var)
    text = alt.hconcat(x_var, y_var)  # Combine data tables

    # Build chart
    chart = alt.hconcat(
                points,
                text
            ).resolve_legend(
            color="independent"
            )
    return chart

In [5]:
# --COMMENT FOR VIEWING THE PLOTS-- The following plots are interactive plots:
#   (1) Selecting the interested region to see the accurate data
#   (2) Moving mouse pointer to any given point to check the brand and model
#       info of the given car

# Plot relationship between Engine Size and CO2 Emissions
plot_one = scatter_plot('Engine Size(L)', 'CO2 Emissions(g/km)')
plot_one

alt.HConcatChart(...)

In [6]:
# Plot relationship between Number of Cylinders and CO2 Emissions
plot_two = scatter_plot('Cylinders', 'CO2 Emissions(g/km)')
plot_two

alt.HConcatChart(...)

In [7]:
# Plot relationship between City Fuel Consumption and CO2 Emissions
plot_three = scatter_plot('Fuel Consumption City(L/100 km)',
                          'CO2 Emissions(g/km)')
plot_three

alt.HConcatChart(...)

In [8]:
# Plot relationship between Highway Fuel Consumption and CO2 Emissions
plot_four = scatter_plot('Fuel Consumption Hwy(L/100 km)',
                         'CO2 Emissions(g/km)')
plot_four

alt.HConcatChart(...)

In [9]:
# Plot relationship between Fuel Consumption(combination) and CO2 Emissions
plot_five = scatter_plot('Fuel Consumption Comb(L/100 km)',
                         'CO2 Emissions(g/km)')
plot_five

alt.HConcatChart(...)

In [10]:
# Plot relationship between Vehicle Classification and CO2 Emissions
plot_six = scatter_plot('Vehicle Class', 'CO2 Emissions(g/km)')
plot_six

alt.HConcatChart(...)

In [11]:
# Develop Machine Learning Model
def learning_model(input_data, model_name, tree_depth=5):
    """
    The function takes in a Pandas DataFrame input_data, a string model_name
    that represents the name for the machine learning model, and if no int
    that represents tree depth of the model is passed in, the default tree
    depth is 5. The function prints and returns the Mean-Squared Error and
    R^2 Score of the model on testing data.
    """
    features = input_data.loc[:, input_data.columns != 'CO2 Emissions(g/km)']
    features = pd.get_dummies(features)
    labels_model = df['CO2 Emissions(g/km)']
    features_train, features_test, labels_train, labels_test = \
        train_test_split(features, labels_model, test_size=0.2)

    # Create an untrained model
    model = DecisionTreeRegressor(max_depth=tree_depth)

    # Train it on the training set
    model.fit(features_train, labels_train)

    # Compute test accuracy
    test_predictions = model.predict(features_test)
    test_mean_squared_error = mean_squared_error(labels_test, test_predictions)
    test_r_squared_score = r2_score(labels_test, test_predictions)
    print('Testing Mean-Squared Error for the' + model_name + ':',
          test_mean_squared_error)
    print('Testing R^2 Score for the:' + model_name + ':',
          test_r_squared_score)

    return test_mean_squared_error, test_r_squared_score


# Construct model with all the given features appropriate for prediction
print()
print('----General Model with all the given features----')
data = df[['Brand', 'Vehicle Class', 'Engine Size(L)', 'Fuel Type',
           'Transmission', 'Fuel Consumption City(L/100 km)',
           'Fuel Consumption Hwy(L/100 km)', 'Cylinders',
           'Fuel Consumption Comb(L/100 km)', 'CO2 Emissions(g/km)',
           'CO2 Rating', 'Smog Rating']]
model_general = learning_model(data, ' General Model')

# Explore on relationship between decision tree and max_depth
print()
tree_r_squared_score = [0] * 10
print('----Explore on relationship between decision tree accuracy and '
      'max_depth:----')
# Simulate models for 10 times
for j in range(10):
    depth = j + 1
    print('Decision Tree Regressor with max_depth', depth)
    tree_tuple = learning_model(data, ' General Model', depth)
    tree_r_squared_score[j] = tree_tuple[1]
    print()


----General Model with all the given features----
Testing Mean-Squared Error for the General Model: 121.44653414053788
Testing R^2 Score for the: General Model: 0.9666652239690023

----Explore on relationship between decision tree accuracy and max_depth:----
Decision Tree Regressor with max_depth 1
Testing Mean-Squared Error for the General Model: 1522.6897980155711
Testing R^2 Score for the: General Model: 0.6137632897480259

Decision Tree Regressor with max_depth 2
Testing Mean-Squared Error for the General Model: 517.012816576665
Testing R^2 Score for the: General Model: 0.8512582920898739

Decision Tree Regressor with max_depth 3
Testing Mean-Squared Error for the General Model: 384.9521904049076
Testing R^2 Score for the: General Model: 0.8981462298767242

Decision Tree Regressor with max_depth 4
Testing Mean-Squared Error for the General Model: 100.53829128939167
Testing R^2 Score for the: General Model: 0.9771457938798691

Decision Tree Regressor with max_depth 5
Testing Mean-S

In [12]:
# Put the calculated data in a dataframe for plotting
depth_list = list(range(1,11))
max_depth = {'Max_depth': depth_list, 'R-squared Score': tree_r_squared_score}
max_depth  = pd.DataFrame.from_dict(max_depth)
display(max_depth)

alt.Chart(max_depth).mark_line().encode(
    x='Max_depth',
    y=alt.Y('R-squared Score', scale=alt.Scale(domain=(0.4, 1.0)))
).properties(
    title='R-squared Score of Max_depth'
)

,Max_depth,R-squared Score
0,1,0.613763
1,2,0.851258
2,3,0.898146
3,4,0.977146
4,5,0.989987
5,6,0.962347
6,7,0.998444
7,8,0.982374
8,9,0.991864
9,10,0.990726


alt.Chart(...)

In [13]:
# Explore on individual feature
feature_list = ['Brand', 'Vehicle Class', 'Engine Size(L)',
                'Cylinders', 'Transmission', 'Fuel Type',
                'Fuel Consumption City(L/100 km)',
                'Fuel Consumption Hwy(L/100 km)',
                'Fuel Consumption Comb(L/100 km)', 'CO2 Rating',
                'Smog Rating']
mean_squared_error_stat = []
r_squared_score_stat = []
model_num = 0
feature_name = ''

for feature in feature_list:
    model_num += 1
    print(feature)
    feature_name = 'Feature ' + feature
    data = df[[feature, 'CO2 Emissions(g/km)']]
    print('----Model # ' + str(model_num) + ' with ' + feature_name + '----')
    my_tuple = learning_model(data, feature_name)
    mean_squared_error_stat.append(my_tuple[0])
    r_squared_score_stat.append(my_tuple[1])
    print()

Brand
----Model # 1 with Feature Brand----
Testing Mean-Squared Error for theFeature Brand: 3503.718471493326
Testing R^2 Score for the:Feature Brand: 0.23675491109993596

Vehicle Class
----Model # 2 with Feature Vehicle Class----
Testing Mean-Squared Error for theFeature Vehicle Class: 3452.551627746956
Testing R^2 Score for the:Feature Vehicle Class: 0.19497146265534748

Engine Size(L)
----Model # 3 with Feature Engine Size(L)----
Testing Mean-Squared Error for theFeature Engine Size(L): 2413.2972087025914
Testing R^2 Score for the:Feature Engine Size(L): 0.3828694881884318

Cylinders
----Model # 4 with Feature Cylinders----
Testing Mean-Squared Error for theFeature Cylinders: 1428.7045074159334
Testing R^2 Score for the:Feature Cylinders: 0.6901702010209917

Transmission
----Model # 5 with Feature Transmission----
Testing Mean-Squared Error for theFeature Transmission: 3572.076495186855
Testing R^2 Score for the:Feature Transmission: 0.18815489201999958

Fuel Type
----Model # 6 with

In [14]:
#  Put the calculated data in a dataframe for plotting
data_features = {'Feature Name': feature_list, 'Mean-Squared Error':
                 mean_squared_error_stat, 'R-squared Score':
                 r_squared_score_stat}
data_features = pd.DataFrame.from_dict(data_features)
display(data_features)

# plot the Mean-Squared Errors of each Features
alt.Chart(data_features).mark_bar().encode(
    x='Mean-Squared Error:Q',
    y=alt.Y('Feature Name:N', sort='-x')
).properties(
    title='Mean-Squared Error of Each Feature',
    width=500,
    height=300)

,Feature Name,Mean-Squared Error,R-squared Score
0,Brand,3503.718471,0.236755
1,Vehicle Class,3452.551628,0.194971
2,Engine Size(L),2413.297209,0.382869
3,Cylinders,1428.704507,0.690170
4,Transmission,3572.076495,0.188155
5,Fuel Type,3685.125730,0.143386
6,Fuel Consumption City(L/100 km),319.927624,0.915429
7,Fuel Consumption Hwy(L/100 km),1010.970323,0.805090
8,Fuel Consumption Comb(L/100 km),120.943212,0.961796
9,CO2 Rating,370.492650,0.891950


alt.Chart(...)

In [15]:
# plot the R-squared Score of each Features
alt.Chart(data_features).mark_bar().encode(
    x='R-squared Score',
    y=alt.Y('Feature Name:N', sort='x')
).properties(
    title='R-squared Score of Each Feature',
    width=500,
    height=300
).configure_mark(
    opacity=0.9,
    color='green'
)

alt.Chart(...)

In [16]:
# Explore on some combinations of features
print('----Combinations of features using three Fuel Consumption '
      'Features(City, Hwy, Comb)----')
features_fuel_consumption = df[['Fuel Consumption City(L/100 km)',
                                'Fuel Consumption Hwy(L/100 km)',
                                'Fuel Consumption Comb(L/100 km)',
                                'CO2 Emissions(g/km)']]
learning_model(features_fuel_consumption, ' Model with Fuel Consumption '
               'features')
print()
print('----Combinations of all features but without using three Fuel '
      'Consumption Features(City, Hwy, Comb)----')
features_low = df[['Brand', 'Vehicle Class', 'Engine Size(L)', 'Cylinders',
                   'Transmission', 'Fuel Type', 'CO2 Rating', 'Smog Rating',
                   'CO2 Emissions(g/km)']]
learning_model(features_low, ' Model without Fuel Consumption features')


----Combinations of features using three Fuel Consumption Features(City, Hwy, Comb)----
Testing Mean-Squared Error for the Model with Fuel Consumption features: 137.93356916770537
Testing R^2 Score for the: Model with Fuel Consumption features: 0.9655575129025796

----Combinations of all features but without using three Fuel Consumption Features(City, Hwy, Comb)----
Testing Mean-Squared Error for the Model without Fuel Consumption features: 200.23212270046338
Testing R^2 Score for the: Model without Fuel Consumption features: 0.946617628076837


(200.23212270046338, 0.946617628076837)